In [3]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

df = pd.read_csv("C:/Users/stive/Documents/Tweets.csv")
df.head()
df.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

In [4]:
tweet_df = df[['text','airline_sentiment']]
print(tweet_df.shape)
tweet_df.head(5)
tweet_df["airline_sentiment"].value_counts()

(14640, 2)


negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [5]:
sentiment_label = tweet_df.airline_sentiment.factorize()
sentiment_label

(array([0, 1, 0, ..., 0, 2, 0], dtype=int64),
 Index(['neutral', 'positive', 'negative'], dtype='object'))

In [6]:
tweet = tweet_df.text.values
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweet)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(tweet)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [7]:
print(tokenizer.word_index)

{'to': 1, 'the': 2, 'i': 3, 'a': 4, 'united': 5, 'you': 6, 'for': 7, 'flight': 8, 'on': 9, 'and': 10, 'my': 11, 'usairways': 12, 'americanair': 13, 'is': 14, 'in': 15, 'southwestair': 16, 'jetblue': 17, 'of': 18, 'me': 19, 'it': 20, 'your': 21, 'have': 22, 'was': 23, 'not': 24, 'with': 25, 'no': 26, 'that': 27, 'at': 28, 'this': 29, 'get': 30, 'but': 31, 't': 32, 'co': 33, 'be': 34, 'from': 35, 'http': 36, 'can': 37, 'are': 38, 'thanks': 39, 'cancelled': 40, 'we': 41, 'now': 42, 'an': 43, 'just': 44, 'service': 45, 'do': 46, 'so': 47, '2': 48, 'help': 49, 'been': 50, 'time': 51, 'will': 52, 'customer': 53, 'up': 54, 'out': 55, 'our': 56, 'they': 57, "i'm": 58, 'amp': 59, 'hours': 60, 'us': 61, 'what': 62, 'when': 63, 'flights': 64, 'how': 65, 'hold': 66, 'plane': 67, 'if': 68, 'all': 69, 'why': 70, 'thank': 71, 'still': 72, 'there': 73, 'one': 74, 'please': 75, 'need': 76, 'would': 77, 'delayed': 78, "can't": 79, 'back': 80, 'virginamerica': 81, 'gate': 82, 'call': 83, 'about': 84, 'ha

In [8]:
print(tweet[0])
print(encoded_docs[0])

@VirginAmerica What @dhepburn said.
[81, 62, 226]


In [9]:
print(padded_sequence[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  81
  62 226]


In [10]:
#Define Layers of the LSTM model
embedding_vector_length = 32
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])  
print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 32)           504608    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 200, 32)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                16600     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 521,259
Trainable params: 521,259
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)

Epoch 1/5
366/366 [==============================] - 185s 471ms/step - loss: -6.3443 - accuracy: 0.1711 - val_loss: -14.9494 - val_accuracy: 0.1226
Epoch 2/5
366/366 [==============================] - 192s 526ms/step - loss: -13.8963 - accuracy: 0.1711 - val_loss: -24.7615 - val_accuracy: 0.1226
Epoch 3/5
366/366 [==============================] - 235s 641ms/step - loss: -20.6507 - accuracy: 0.1711 - val_loss: -34.3804 - val_accuracy: 0.1226
Epoch 4/5
366/366 [==============================] - 186s 508ms/step - loss: -27.2574 - accuracy: 0.1711 - val_loss: -43.8755 - val_accuracy: 0.1226
Epoch 5/5
366/366 [==============================] - 185s 507ms/step - loss: -33.8827 - accuracy: 0.1711 - val_loss: -53.3535 - val_accuracy: 0.1226


In [12]:
def predict_sentiment(text):
    tw = tokenizer.texts_to_sequences([text])
    tw = pad_sequences(tw,maxlen=200)
    prediction = int(model.predict(tw).round().item())
    print("Predicted label: ", sentiment_label[1][prediction])

In [13]:
test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)

test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)

Predicted label:  positive
Predicted label:  positive
